## Import Pandas for reading a csv

In [8]:
!pip install -q pandas

ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement snowflake-snowpark-python<2,>=1.5.1, but you'll have snowflake-snowpark-python 1.5.0 which is incompatible.
ERROR: snowflake-connector-python 3.7.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
ERROR: botocore 1.34.51 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Import Snowpark Pandas for dumping csv to Snowflake table

In [7]:
!pip install "snowflake-snowpark-python[pandas]"

     |████████████████████████████████| 419kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 829kB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 2.6MB 52.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 747kB 73.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 85.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 92.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 85.5MB/s eta 0:00:01
     |████████████████████████████████| 450kB 81.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 22.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl
     |████████████████████████████████| 153kB 82.1MB/s eta 0:00:01
     |████████████████████████████████| 4.6MB

In [2]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

In [4]:
df =  pd.read_csv("/notebooks/notebooks/customer_data.csv")

In [7]:
df.head()

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000
3,1,58995168,7,1335.5,1,151000.0,2,11,5,110,3,60084,1.000000
4,0,54987320,7,NaN,2,59000.0,2,11,5,108,4,450081,197.403141


In [6]:
df.isnull().sum()

label       0
id          0
fea_1       0
fea_2     149
fea_3       0
fea_4       0
fea_5       0
fea_6       0
fea_7       0
fea_8       0
fea_9       0
fea_10      0
fea_11      0
dtype: int64

## Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [9]:
from snowflake.snowpark.session import Session

connection_params = {
    'user': 'MANISH',
    'password': 'Password@2023',
    'account': 'ug94937.us-east4.gcp',
    'warehouse': 'FOSFOR_FDC',
    'database': 'FDC_DATA_MANISH',
    'schema': 'PUBLIC',
    'role': 'MANISH'
}
session = Session.builder.configs(connection_params).create()
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/customer_data.csv")
df.fillna(0, inplace=True)
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA")

In [15]:
df =  pd.read_csv("/notebooks/notebooks/payment_data.csv")
df.head()

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016


In [16]:
df.isnull().sum()

id                    0
OVD_t1                0
OVD_t2                0
OVD_t3                0
OVD_sum               0
pay_normal            0
prod_code             0
prod_limit         6118
update_date          26
new_balance           0
highest_balance     409
report_date        1114
dtype: int64

In [24]:
df[['prod_limit','highest_balance']] = df[['prod_limit','highest_balance']].fillna(0)
df[['update_date','report_date']] = df[['prod_limit','highest_balance']].fillna('31/12/9999')

In [25]:
df.isnull().sum()

id                 0
OVD_t1             0
OVD_t2             0
OVD_t3             0
OVD_sum            0
pay_normal         0
prod_code          0
prod_limit         0
update_date        0
new_balance        0
highest_balance    0
report_date        0
dtype: int64

## Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [26]:
from snowflake.snowpark.session import Session

connection_params = {
    'user': 'MANISH',
    'password': 'Password@2023',
    'account': 'ug94937.us-east4.gcp',
    'warehouse': 'FOSFOR_FDC',
    'database': 'FDC_DATA_MANISH',
    'schema': 'PUBLIC',
    'role': 'MANISH'
}
session = Session.builder.configs(connection_params).create()
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/payment_data.csv")
df[['prod_limit','highest_balance']] = df[['prod_limit','highest_balance']].fillna(0)
df[['update_date','report_date']] = df[['prod_limit','highest_balance']].fillna('31/12/9999')
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CC_PAYMENT_DATA")

In [37]:
!pip install snowflake-snowpark-python

  Using cached https://files.pythonhosted.org/packages/e8/64/78013f54adc402424e980c94c7e8c9d69dc4648649796c8b2ba8fdddb09b/snowflake_snowpark_python-1.14.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/15/80/44286939ca215e88fa827b2aeb6fa3fd2b4a7af322485c7170d6f9fd96e0/cloudpickle-2.2.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/c7/03d2ca5e460acb3a31d70ee17584fbc6fe828a52c9f83bbb20a392b8988b/snowflake_connector_python-3.7.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/92/e1/1c8bb3420105e70bdf357d57dd5567202b4ef8d27f810e98bb962d950834/setuptools-69.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7d/39/472f2554a0f1e825bd7c5afc11c817cd7a2f3657460f7159f691fbb37c51/PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b4326981

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': 'MANISH',
    'password': 'Password@2023',
    'account': 'ug94937.us-east4.gcp',
    'warehouse': 'FOSFOR_FDC',
    'database': 'FDC_DATA_MANISH',
    'schema': 'PUBLIC',
    'role': 'MANISH'
}
session = Session.builder.configs(connection_params).create()
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

cc_data = session.table('FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA')

In [2]:
cc_data.to_pandas()

SnowparkFetchDataException: (1406): Failed to fetch a pandas Dataframe. The error is: 255002: Optional dependency: 'pandas' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

AttributeError: 'function' object has no attribute 'show'